# Homework 3, Exercise 3 refers to this IPython notebook

## The content of the problem is on the problem set. 

In [ ]:
from HW3aux import *
from random import choice

Let's recall some of the sorting algorithms we've seen so far.

### Note: 
All of these algorithms (except radixSort) have been modified to sort not general numbers but objects of class "myInt" which is defined in HW3aux.py.  This allows them to be used as subroutines in radixSort.  The implementaiton of radixSort itself given below works on normal Python ints.

### Here is our implementation of bucketSort from Lecture 6

In [ ]:
# A is a list to sort.
# A contains objects that have a "key" attribute, where "key" is between 0 and bucketMax - 1
def bucketSort(A, bucketMax=10):  
    # myQueue is a simple FIFO queue class defined in HW3aux.py.  
    # Initialize a myQueue for each bucket.
    T = [ myQueue() for i in range(bucketMax) ] 
    for x in A:
        # add object x to the appropriate bucket
        T[x.key()].push(x)
    # now return the concatenated buckets.
    ret = []
    for i in range(bucketMax):
        ret += T[i].getList()
    return ret

### Here is one of our implementations of quickSort from Lecture 5

In [ ]:
# A is a list to sort.
# A contains objects that have a "key" attribute, and we will sort according to that attribute.
def quickSort(A):
    return quickSortHelper(A, 0, len(A))
        
def quickSortHelper(A, start, end):
    if end - start <= 1:
        return 
    # choose a random pivot:
    p = choice(range(start, end)) 
    # partition around the pivot:
    pivotLocation = partition( A, start, end, p ) 
    # recurse:
    quickSortHelper(A,start, pivotLocation)
    quickSortHelper(A,pivotLocation + 1, end)
    return

# Helper function to swap A[i] with A[j]
def swap(A, i, j):
    tmp = A[i]
    A[i] = A[j]
    A[j] = tmp

# Here's one in-place Partition algorithm. (This is the one in CLRS.)
# It rearranges A[start:end] around the pivot p.
# It returns the index where A[p] ended up.
def partition(A, start, end, p):
    # first put the pivot at the end
    x = A[p]
    swap(A, p, end-1)
    # now do the partition algorithm we went over in class
    i = start - 1
    for j in range(start, end-1):
        if A[j].key() <= x.key():
            i += 1
            swap(A, i, j)
    # at this point all of the things at positions <i are smaller than the pivot, and at >=i are larger than the pivot.
    # so put the pivot back where it needs to go:
    swap(A, i+1, end-1)
    return i+1

### Here is our implementation of MergeSort from Lecture 2

We've tweaked it a little bit to allow you to choose between two possible Merge functions.

merge1 is the version we saw in class.  If the pointers in the two halves are pointing at something with the same value, we'll add the one from R before we add the one from L.

merge2 is *almost* the same as merge1: only one < has been changed to a <=.  This means that, in the situation above, we'll add an element from L before we add one from R if they are equal.

In [ ]:
# mergeSort takes a list A to sort.
# objects in A have a "key" attribute, and we will sort according to that attribute
# whichMerge is an optional argument with default 1.  
# If whichMerge=1, then we use merge1 below.  Otherwise we use merge2.
def mergeSort(A,whichMerge=1):
    n = len(A)
    if n <= 1:
        return A
    L = mergeSort(A[:round(n/2)], whichMerge)
    R = mergeSort(A[round(n/2):n], whichMerge)
    if whichMerge == 1:
        return merge1(L,R)
    else:
        return merge2(L,R)

# This is the merge algorithm we saw in class.
def merge1(L, R):
    i = 0 # current index in the L array
    j = 0 # current index in the R array
    ret = []
    while i < len(L) and j < len(R):
        if L[i].key() < R[j].key():
            ret.append(L[i])
            i += 1
        else:    
            ret.append(R[j])
            j += 1
    while i < len(L):
        ret.append(L[i])
        i += 1
    while j < len(R):
        ret.append(R[j])
        j+= 1
    return ret

# This is only very slightly different than the algorithm we saw in class
def merge2(L, R):
    i = 0 # current index in the L array
    j = 0 # current index in the R array
    ret = []
    while i < len(L) and j < len(R):
        # The following is the ONLY place where merge1 and merge2 differ.  
        # There is a <= instead of a <.
        if L[i].key() <= R[j].key():
            ret.append(L[i])
            i += 1
        else:     
            ret.append(R[j])
            j += 1
    while i < len(L):
        ret.append(L[i])
        i += 1
    while j < len(R):
        ret.append(R[j])
        j+= 1
    return ret

### Here is our implementation RadixSort from Lecture 6

This is the method that you will have to modify to change the inner sorting algorithms.  They are already in there, commented out.

In [ ]:
# radixSort sorts integers which can be written with length <nDigits> in in base <base> 
# notice that if the base isn't 10 we probably shouldn't call these "digits," but oh well...
def radixSort(A, nDigits, base=10):
    # replace A with a list of "myInts" (from the lecture 6 auxiliary file).
    B = [ myInt(x, base=base) for x in A ]
    # now repeatedly run another sorting algorithm for each digit.
    for j in range(nDigits):
        # first update the digit that is the key
        for x in B:
            x.updateKeyDigit(j)
        #############
        #
        # TODO: we want to sort on that digit; what happens if we use another sorting algorithm like quickSort or mergeSort?
        #
        # Choose which of the four algorithms below are commented out and see how it works!
        #
        ###
        # Option 1: BucketSort:
        B = bucketSort(B,bucketMax=base)
        # Option 2: QuickSort:
        #quickSort(B)
        # Option 3: MergeSort with the "merge" algorithm we saw in class.
        #B = mergeSort(B, whichMerge=1)
        # Option 4: MergeSort with a very slightly modified merge algorithm.
        #B = mergeSort(B, whichMerge=2)
        #
        ###############
    # now B should be sorted!  We hope...
    return [b.getValue() for b in B]  

## Now let's see how RadixSort does with different inner sorting algorithms.

Here's some code to try it out (we'll keep the base as 10 for this exercise, since it's easier to see what's going on).

### In problem 3, you will have to run this with all of the different inner sorting algorithms.  Which ones work?  Which ones don't?

In [ ]:
# A is a list of integers
# Returns True if B is sorted and False otherwise
def isSorted(A):
    for i in range(len(A)-1):
        if A[i] > A[i+1]:
            return False
    return True

In [ ]:
# Test radixSort on <nTrials> different random lists of length <n>.
# The lists have numbers that are <nDigits> long in base <base>
# Returns True if radixSort works properly on all of the trials.
#
# All of the parameters are optional and the defaults should be fine for this exercise
#
# Warning: Python may freak out if you start generating numbers on the order of 10^(19) or so. 
# (aka if you set nDigits>=19).
#
def testRadixSort(nDigits=5,n=100,base=10,nTrials=50):
    for trial in range(nTrials):
        # generate a random list of b nDigits-long integers (in base <base>).
        A = [ choice( range(base**(nDigits)) ) for i in range(n)]
        # does it work?
        if not isSorted(radixSort(A, nDigits, base)):
            print("Nope, that one doesn't work.")
            return False
    print("Seems to work!")
    return True

In [ ]:
testRadixSort()